# Advent of Code 2021 - Day 14

In [275]:
data class Input(val polymer: String, val rules: Map<String, String>)

In [276]:
import java.io.File

val input: Input = File("Day14.input.txt")
    .bufferedReader()
    .readLines()
    .let { lines ->
        val insertionPairRegex = Regex("""(\w{2}) -> (\w)""")
        Input(
            polymer = lines[0],
            rules = lines.drop(2)
                .map { insertionPairRegex.find(it)!!.destructured }
                .associate { (pair, insert) -> pair to insert }
        )
    }

## Part 1

Given an `Input::polymer` insert the character defined by `Input::rules`' value in between each occurrence of `Input::rules`' keys. Each insertion happens simultaneously. For instance, given the polymer "NNCB" and the rules: NN -> C, NC -> B, CB -> H, the final polymer would be "NCNBCHB". Find the difference between the maximum occurrence of each character with the minimum after 10 iterations of insertions.

In [277]:
fun String.insert(rules: Map<String, String>): String =
    windowed(2)
    .map { "${it[0]}${rules.getOrDefault(it, "")}${it[1]}" }
    .let { segments ->
        segments.dropLast(1).map { it.dropLast(1) } + segments.last()
    }
    .joinToString("")

tailrec fun String.expand(rules: Map<String, String>, times: Int): String =
    if (times == 0) this else this.insert(rules).expand(rules, times - 1)

fun String.minMaxDifference(): Int = 
    groupingBy { it }
    .eachCount()
    .let { it.maxByOrNull { it.value }!!.value - it.minByOrNull { it.value }!!.value }

input.polymer.expand(input.rules, 10)
    .minMaxDifference()

3697

### Notes

This is the "brute force" approach which follows the implied solution of the problem statement, which is to maintain the entire string through each iteration. After all iterations, we can just count the number of each character for the answer.

## Part 2

Same as part 1, but with 40 iterations.

In [278]:
fun <T> MutableMap<T, Long>.increment(key: T, value: Long = 1) {
    when (val count = get(key)) {
        null -> this[key] = value
        else -> this[key] = count + value
    }
}

fun Map<String, Long>.insert(rules: Map<String, String>): Map<String, Long> =
    entries.fold(mutableMapOf()) { acc, (pair, count) ->
        acc.apply {
            rules[pair]?.let {
                increment("${pair[0]}$it", count)
                increment("$it${pair[1]}", count)
            }
        }
    }

fun String.explode(rules: Map<String, String>, times: Int): Map<String, Long> {
    tailrec fun Map<String, Long>.decrement(t: Int): Map<String, Long> = 
        if (t == 0) this@decrement else this@decrement.insert(rules).decrement(t - 1)
    
    return this@explode.windowed(2).fold(mutableMapOf<String, Long>()) { acc, pair ->
        acc.apply { increment(pair) }
    }.decrement(times)
}

fun Map<String, Long>.minMaxDifference(): Long = 
    entries.fold(mutableMapOf<Char, Long>()) { acc, (pair, count) ->
        acc.apply { increment(pair[1], count) }
    }
    .let { map -> map.maxByOrNull { it.value }!!.value - map.minByOrNull { it.value }!!.value }

input.polymer.explode(input.rules, 40).minMaxDifference()

4371307836157

### Notes

Maintaining the entire string for 40 iterations is way too long, so a new solution had to be written (which also solves for part 1). The key here is to notice that we don't care about the entire string, but rather, just the pairs within them and how many times those pairs show up in the string. So from iteration to iteration, we can just pass in the counts of each pair and generate new pairs. This is the first part of the problem.

Now with the counts of pairs, we have to get the number of times each character would show up as if we had the entire string. The key thing to notice here is that, if we knew the order, to build the string we have to connect each pair end-to-end with a matching pair: AB <-> BC. This means that we can just count the occurrences of the last character of each pair to find our answer.